## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Importando o dataset e o transformando em DataFrame do Pandas

In [2]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
chipo = pd.read_table(url)

## Analisando informações gerais

In [3]:
chipo.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [4]:
chipo.describe()

,order_id,quantity
count,4622.000000,4622.000000
mean,927.254868,1.075725
std,528.890796,0.410186
min,1.000000,1.000000
25%,477.250000,1.000000
50%,926.000000,1.000000
75%,1393.000000,1.000000
max,1834.000000,15.000000


In [5]:
chipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [6]:
chipo.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


#### Já é possível observar que a coluna item_price teria que ser transformada em uma varável numérica para podermos trabalhar com ela.

Verificar se a quantidade afeta o valor da compra

Verificando os itens que tem mais de uma quantidade

In [7]:
chipo_aux = chipo.copy() # Fazendo uma cópia do DataFrame

In [8]:
aux = chipo[chipo['quantity']>1][['item_name', 'choice_description']].reset_index(drop=True)
aux = pd.DataFrame(aux)
aux = aux.drop_duplicates().reset_index(drop=True)
aux

,item_name,choice_description
0,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans..."
1,Canned Soda,[Sprite]
2,Canned Soda,[Mountain Dew]
3,Chicken Salad Bowl,"[Tomatillo Green Chili Salsa, [Sour Cream, Che..."
4,Steak Burrito,"[Tomatillo-Red Chili Salsa (Hot), [Rice, Chees..."
...,...,...
116,Steak Soft Tacos,"[Fresh Tomato Salsa, [Cheese, Lettuce]]"
117,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Fajita Vegetables,..."
118,Steak Bowl,"[Fresh Tomato Salsa, [Rice, Pinto Beans, Sour ..."
119,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Pinto Beans, Chees..."


In [9]:
chipo_aux[['item_name','choice_description']]

,item_name,choice_description
0,Chips and Fresh Tomato Salsa,NaN
1,Izze,[Clementine]
2,Nantucket Nectar,[Apple]
3,Chips and Tomatillo-Green Chili Salsa,NaN
4,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans..."
...,...,...
4617,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ..."
4618,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese..."
4619,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto..."
4620,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu..."


In [10]:
chipo_aux = chipo_aux[(chipo_aux['item_name'].isin(aux['item_name'])) & (chipo_aux['choice_description'].isin(aux['choice_description']))] 
# Tabela só com os que tem pelo menos um pedido com mais de uma quantidade
chipo_aux.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
6,3,1,Side of Chips,NaN,$1.69
10,5,1,Chips and Guacamole,NaN,$4.45


In [11]:
chipo_aux = chipo_aux.pivot_table(values='order_id', index=['item_name', 'choice_description'], aggfunc='count', columns='quantity').reset_index()
# Criando tabela para conseguir verificar quais elementos são possíveis de verificar se há relação entre a quantidade e o preço
chipo_aux.head()

quantity,item_name,choice_description,1,2,3,4
0,6 Pack Soft Drink,[Coke],21.0,NaN,NaN,NaN
1,6 Pack Soft Drink,[Diet Coke],23.0,1.0,NaN,NaN
2,6 Pack Soft Drink,[Lemonade],2.0,NaN,NaN,NaN
3,6 Pack Soft Drink,[Sprite],5.0,NaN,NaN,NaN
4,Barbacoa Crispy Tacos,"[Fresh Tomato Salsa, [Rice, Cheese, Sour Cream]]",1.0,NaN,NaN,NaN


In [12]:
chipo_aux['contagem_aux'] = chipo_aux[range(1,5)].count(axis=1)
chipo_aux = chipo_aux.iloc[:,[0,1,-1]]
chipo_aux = chipo_aux[chipo_aux['contagem_aux'] > 1].reset_index(drop=True)
# Criando uma coluna para ver quais tem valores para 'quantity' diferentes para poder verificar a  relação entre quantidade e preço

In [13]:
aux = chipo_aux.loc[0,:]
print(aux[0])
print(aux[1])
print()
print(aux)

6 Pack Soft Drink
[Diet Coke]

quantity
item_name             6 Pack Soft Drink
choice_description          [Diet Coke]
contagem_aux                          2
Name: 0, dtype: object


In [14]:
chipo_aux = chipo.copy()

In [15]:
chipo_aux['item_price'] = (chipo_aux['item_price'].str.strip('$')).astype('float64') # Transformando a coluna 'item_price' em float

In [16]:
chipo_aux = chipo_aux[(chipo_aux['item_name'] == aux[0]) & (chipo_aux['choice_description'] == aux[1])]
chipo_aux.loc[:,'item_price'] = chipo_aux['item_price']/chipo_aux['quantity']
print(len(chipo_aux['item_price'].unique()))

1


In [17]:
print(chipo_aux['item_price'].unique())
print(len(chipo_aux['item_price'].unique()))

[6.49]
1


#### Quando todos os valores foram divididos pela quantidade do pedido, os preços foram iguais, logo é possível concluir que o preço do pedido é relacionado com a quantidade

Verificando quantas vezes cada item da coluna 'item_name' estão associados com quantos itens da coluna 'choice_description'

In [18]:
aux1 = chipo.pivot_table(index='choice_description', values='item_price', columns='item_name', aggfunc='count').reset_index()
aux1.head()

item_name,choice_description,6 Pack Soft Drink,Barbacoa Bowl,Barbacoa Burrito,Barbacoa Crispy Tacos,Barbacoa Salad Bowl,Barbacoa Soft Tacos,Bowl,Burrito,Canned Soda,...,Steak Crispy Tacos,Steak Salad,Steak Salad Bowl,Steak Soft Tacos,Veggie Bowl,Veggie Burrito,Veggie Crispy Tacos,Veggie Salad,Veggie Salad Bowl,Veggie Soft Tacos
0,"[Adobo-Marinated and Grilled Chicken, Pinto Be...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Adobo-Marinated and Grilled Chicken, [Sour Cr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[Adobo-Marinated and Grilled Chicken],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Adobo-Marinated and Grilled Steak, [Sour Crea...",NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[Adobo-Marinated and Grilled Steak],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Criando uma coluna que tenha a contagem de quantos elementos de 'item_price' se repetem para a 'choice_description'

In [19]:
aux1['contagem'] = aux1.count(axis=1) - 1

In [20]:
aux1 = aux1[['choice_description', 'contagem']]

In [21]:
aux1.head()

item_name,choice_description,contagem
0,"[Adobo-Marinated and Grilled Chicken, Pinto Be...",1
1,"[Adobo-Marinated and Grilled Chicken, [Sour Cr...",1
2,[Adobo-Marinated and Grilled Chicken],1
3,"[Adobo-Marinated and Grilled Steak, [Sour Crea...",1
4,[Adobo-Marinated and Grilled Steak],1


In [22]:
aux1[aux1['contagem'] > 1]

item_name,choice_description,contagem
14,[Coke],2
15,[Diet Coke],3
19,"[Fresh Tomato (Mild), [Black Beans, Rice, Sour...",2
31,"[Fresh Tomato (Mild), [Lettuce, Black Beans, R...",2
42,"[Fresh Tomato Salsa (Mild), Black Beans]",2
...,...,...
1015,"[[Tomatillo-Green Chili Salsa (Medium), Roaste...",2
1020,"[[Tomatillo-Green Chili Salsa (Medium), Roaste...",2
1023,"[[Tomatillo-Green Chili Salsa (Medium), Roaste...",2
1024,"[[Tomatillo-Green Chili Salsa (Medium), Roaste...",2


#### Com a tabela acima é possível observar, que existem descrições que são iguais para mais de um tipo de item, logo é possível chegar a conclusão que para analisar quantas possibilidades diferente podem existir de pedidos, é necessário observar as duas colunas juntamente

## Tratando das colunas

In [23]:
chipo_aux = chipo.copy() # Fazendo uma cópia da célula

### Criando uma célulda que tenha a coluna 'item_name' e a coluna 'choice_description' juntas

In [24]:
chipo_aux.loc[chipo_aux['choice_description'].isna()==False, 'name_description'] = chipo_aux['item_name'] + chipo_aux['choice_description']
chipo_aux.loc[chipo_aux['choice_description'].isna(), 'name_description'] = chipo_aux['item_name']

In [25]:
chipo_aux.head()

,order_id,quantity,item_name,choice_description,item_price,name_description
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,Chips and Fresh Tomato Salsa
1,1,1,Izze,[Clementine],$3.39,Izze[Clementine]
2,1,1,Nantucket Nectar,[Apple],$3.39,Nantucket Nectar[Apple]
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,Chips and Tomatillo-Green Chili Salsa
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,"Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ..."


### Tratando do tipo da coluna de preço

In [26]:
chipo_aux['order_price'] = chipo_aux['item_price'].str.split('$', expand=True)[1].astype('float64')
chipo_aux.drop(columns='item_price', inplace=True)

In [27]:
chipo_aux.head()

,order_id,quantity,item_name,choice_description,name_description,order_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,Chips and Fresh Tomato Salsa,2.39
1,1,1,Izze,[Clementine],Izze[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],Nantucket Nectar[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,Chips and Tomatillo-Green Chili Salsa,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...","Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ...",16.98


### Criando uma coluna com o preço unitário dos itens

In [28]:
chipo_aux.loc[:,'unit_price'] = (chipo_aux['order_price']/chipo_aux['quantity']).round(2)


In [29]:
chipo_aux.head()

,order_id,quantity,item_name,choice_description,name_description,order_price,unit_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,Chips and Fresh Tomato Salsa,2.39,2.39
1,1,1,Izze,[Clementine],Izze[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],Nantucket Nectar[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,Chips and Tomatillo-Green Chili Salsa,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...","Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ...",16.98,8.49


#### Salvando o que já foi feito

In [30]:
chipo = chipo_aux.copy()

### Verificando se existe mais de um preço para o mesmo produto

In [31]:
chipo_aux = chipo[['name_description','unit_price']]

In [32]:
chipo_aux.head()

,name_description,unit_price
0,Chips and Fresh Tomato Salsa,2.39
1,Izze[Clementine],3.39
2,Nantucket Nectar[Apple],3.39
3,Chips and Tomatillo-Green Chili Salsa,2.39
4,"Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ...",8.49


In [33]:
chipo_aux = chipo_aux.drop_duplicates()

In [34]:
chipo_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1879 entries, 0 to 4612
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name_description  1879 non-null   object 
 1   unit_price        1879 non-null   float64
dtypes: float64(1), object(1)
memory usage: 44.0+ KB


#### Criando uma tabela com os itens duplicados que tem preços diferentes

In [35]:
diferent_prices = chipo_aux[chipo_aux['name_description'].duplicated() == True]
diferent_prices


,name_description,unit_price
49,Chips and Guacamole,3.99
89,Chips and Fresh Tomato Salsa,2.95
329,Bottled Water,1.50
676,Chips and Guacamole,4.00
730,Chips and Fresh Tomato Salsa,2.29
927,Chips and Guacamole,3.89
4237,Chips and Guacamole,4.25
4509,Chips,1.99


In [36]:
chipo_aux.loc[chipo_aux['name_description'].isin(diferent_prices['name_description'])]

,name_description,unit_price
0,Chips and Fresh Tomato Salsa,2.39
10,Chips and Guacamole,4.45
34,Bottled Water,1.09
40,Chips,2.15
49,Chips and Guacamole,3.99
89,Chips and Fresh Tomato Salsa,2.95
329,Bottled Water,1.50
676,Chips and Guacamole,4.00
730,Chips and Fresh Tomato Salsa,2.29
927,Chips and Guacamole,3.89


#### Verificando mais informações sobre esses itens com variação de preço

In [37]:
aux2 = (chipo_aux.loc[chipo_aux['name_description'].isin(diferent_prices['name_description'])].pivot_table(index='name_description', values='unit_price', aggfunc=['mean','std','min', 'max']))
aux2[['mean','std']] = aux2[['mean','std']].round(2)
aux2

,mean,std,min,max
,unit_price,unit_price,unit_price,unit_price
name_description,,,,
Bottled Water,1.30,0.29,1.09,1.50
Chips,2.07,0.11,1.99,2.15
Chips and Fresh Tomato Salsa,2.54,0.36,2.29,2.95
Chips and Guacamole,4.12,0.23,3.89,4.45


In [38]:
aux2.index

Index(['Bottled Water', 'Chips', 'Chips and Fresh Tomato Salsa',
       'Chips and Guacamole'],
      dtype='object', name='name_description')

In [39]:
price_variation = chipo[chipo['name_description'].isin(aux2.index)].sort_values(['name_description', 'unit_price'])
price_variation.pivot_table(index='name_description', values='order_price', columns='unit_price', aggfunc='count')

unit_price,1.09,1.50,1.99,2.15,2.29,2.39,2.95,3.89,3.99,4.00,4.25,4.45
name_description,,,,,,,,,,,,
Bottled Water,27.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chips,NaN,NaN,1.0,210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chips and Fresh Tomato Salsa,NaN,NaN,NaN,NaN,3.0,36.0,71.0,NaN,NaN,NaN,NaN,NaN
Chips and Guacamole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,99.0,2.0,1.0,371.0


## Questionário

### a) Importe os dados em notebook e atribua à variável chipo.

In [40]:
chipo.head()

,order_id,quantity,item_name,choice_description,name_description,order_price,unit_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,Chips and Fresh Tomato Salsa,2.39,2.39
1,1,1,Izze,[Clementine],Izze[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],Nantucket Nectar[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,Chips and Tomatillo-Green Chili Salsa,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...","Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ...",16.98,8.49


### b) Selecione e mostre as 10 primeiras entradas.

In [41]:
chipo.head(10)

,order_id,quantity,item_name,choice_description,name_description,order_price,unit_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,Chips and Fresh Tomato Salsa,2.39,2.39
1,1,1,Izze,[Clementine],Izze[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],Nantucket Nectar[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,Chips and Tomatillo-Green Chili Salsa,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...","Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ...",16.98,8.49
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...","Chicken Bowl[Fresh Tomato Salsa (Mild), [Rice,...",10.98,10.98
6,3,1,Side of Chips,NaN,Side of Chips,1.69,1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...","Steak Burrito[Tomatillo Red Chili Salsa, [Faji...",11.75,11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...","Steak Soft Tacos[Tomatillo Green Chili Salsa, ...",9.25,9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...","Steak Burrito[Fresh Tomato Salsa, [Rice, Black...",9.25,9.25


### c) Quantas observações estão presentes neste DataSet?

In [42]:
len(chipo)

4622

### d) Qual o número de colunas neste DataSet?

In [43]:
len(chipo.columns)

7

### e) Qual o nome de todas as colunas deste DataSet?

In [44]:
for column in chipo.columns:
    print(column)

order_id
quantity
item_name
choice_description
name_description
order_price
unit_price


### f) Qual o produto mais pedido? Quantas vezes ele foi pedido?

In [45]:
chipo.pivot_table(index='item_name', values='quantity', aggfunc='sum').reset_index().sort_values('quantity', ascending=False).head(1)

,item_name,quantity
17,Chicken Bowl,761


### g) Qual foi o pedido mais pedido dentro da coluna choice_description?

In [46]:
chipo.pivot_table(index='choice_description', values='quantity', aggfunc='sum').reset_index().sort_values('quantity', ascending=False).head(1)

,choice_description,quantity
15,[Diet Coke],159


### h) Quantos itens foram pedidos no total?

In [47]:
len(chipo['item_name'].unique())

50

### i) Qual foi o valor recebido para o período representado nessa base de dados?

In [48]:
chipo['order_price'].sum()

34500.16

### j) Qual a média do valor recebido por pedido?

In [49]:
chipo.pivot_table(index='order_id', values='order_price', aggfunc='sum')['order_price'].mean().round(2)

18.81

### k) Quantos itens diferentes estão esgotados?

### l) Quantos produtos custam mais de $10?


In [50]:
len(chipo.loc[:, ['name_description', 'unit_price']][chipo['unit_price'] < 10].drop_duplicates())

1172

### m) Qual o preço para cada um deles?

In [51]:
chipo.loc[:, ['name_description', 'unit_price']][chipo['unit_price'] < 10].drop_duplicates()

,name_description,unit_price
0,Chips and Fresh Tomato Salsa,2.39
1,Izze[Clementine],3.39
2,Nantucket Nectar[Apple],3.39
3,Chips and Tomatillo-Green Chili Salsa,2.39
4,"Chicken Bowl[Tomatillo-Red Chili Salsa (Hot), ...",8.49
...,...,...
4573,"Veggie Salad Bowl[Fresh Tomato Salsa, [Fajita ...",8.75
4595,"Chicken Bowl[Tomatillo Green Chili Salsa, [Ric...",8.75
4599,"Chicken Bowl[Roasted Chili Corn Salsa, [Cheese...",8.75
4602,Barbacoa Burrito[Tomatillo Green Chili Salsa],9.25


n) Ordene os itens que custam mais de $10 por ordem alfabética.

In [52]:
chipo.loc[:, ['name_description', 'unit_price']][chipo['unit_price'] < 10].drop_duplicates().sort_values(['name_description','unit_price'])

,name_description,unit_price
357,6 Pack Soft Drink[Coke],6.49
341,6 Pack Soft Drink[Diet Coke],6.49
3141,6 Pack Soft Drink[Lemonade],6.49
721,6 Pack Soft Drink[Nestea],6.49
298,6 Pack Soft Drink[Sprite],6.49
...,...,...
3889,"Veggie Soft Tacos[Fresh Tomato Salsa (Mild), [...",8.49
1395,"Veggie Soft Tacos[Fresh Tomato Salsa (Mild), [...",8.49
781,"Veggie Soft Tacos[Fresh Tomato Salsa, [Black B...",8.75
2851,Veggie Soft Tacos[Roasted Chili Corn Salsa (Me...,8.49


### o) Quantas vezes o item mais caro foi pedido?

In [53]:
most_expensives = chipo[chipo['unit_price'] == chipo['unit_price'].max()]['item_name'].unique()
most_expensives

array(['Steak Salad Bowl', 'Carnitas Salad Bowl', 'Barbacoa Salad Bowl'],
      dtype=object)

In [54]:
chipo[chipo['item_name'].isin(most_expensives)].pivot_table(index='item_name', values='quantity', aggfunc='sum').sort_values('quantity', ascending=False)

,quantity
item_name,
Steak Salad Bowl,31
Barbacoa Salad Bowl,10
Carnitas Salad Bowl,6


### p) Quantas vezes uma Veggie Salad Bowl foi pedida?

In [55]:
chipo[chipo['item_name'] == 'Veggie Salad Bowl']['quantity'].sum()

18

### q) Crie um histograma com os 5 produtos mais pedidos.

In [56]:
most_ordered = chipo.loc[:, ['quantity', 'name_description']].pivot_table(index='name_description', values='quantity', aggfunc='sum').sort_values('quantity', ascending=False).head().reset_index()
most_ordered

,name_description,quantity
0,Chips and Guacamole,506
1,Chips,230
2,Bottled Water,211
3,Chips and Fresh Tomato Salsa,130
4,Canned Soft Drink[Coke],122


# Não faz sentido plotar esse gráfico com o histograma e sim com o gráfico de barras

In [57]:
fig = px.bar(most_ordered, x='name_description', y='quantity')

fig.update_layout(
    title='5 produtos mais pedidos', 
    yaxis_title='Quantity',
    xaxis_title='Produtos',
    font={'color':'gray', 'size':18, 'family':'Courier New'},
    title_font={'size':30},
    height=600
)

fig.update_traces(
    hovertemplate='Produto: %{x}<br>Quantidade: %{y}',  # Personalize os nomes das colunas aqui
)

fig.show()

### r) Crie um gráfico de espalhamento onde em um eixo está o número de vezes que um pedido foi realizado e no outro eixo está o preço do produto.


In [58]:
table_aux = chipo.pivot_table(index=['name_description','unit_price'], values='quantity', aggfunc='sum').reset_index().sort_values('quantity', ascending=False)
table_aux

,name_description,unit_price,quantity
1204,Chips and Guacamole,4.45,396
1196,Chips,2.15,229
163,Bottled Water,1.50,177
178,Canned Soft Drink[Coke],1.25,122
179,Canned Soft Drink[Diet Coke],1.25,115
...,...,...,...
966,"Chicken Burrito[[Fresh Tomato Salsa (Mild), Ro...",8.49,1
965,"Chicken Burrito[[Fresh Tomato Salsa (Mild), Ro...",8.49,1
964,"Chicken Burrito[[Fresh Tomato Salsa (Mild), Ro...",8.49,1
963,"Chicken Burrito[[Fresh Tomato Salsa (Mild), Ro...",8.49,1


In [61]:
fig = px.scatter(table_aux, x='quantity', y='unit_price', hover_data='name_description')

fig.update_layout(
    title='Nº de vezes pedido vs. Preço', 
    yaxis_title='Preço (R$)',
    xaxis_title='Vezes que foi pedido',
    font={'color':'gray', 'size':18, 'family':'Courier New'},
    title_font={'size':30},
    height=600,
)

fig.update_xaxes(type='log')

fig.update_traces(
    hovertemplate='%{customdata[0]}<br>Vezes pedido: %{x}<br>Preço por unidade: R$%{y}',  # Personalize os nomes das colunas aqui
)
fig.show()

### s)Baseado no gráfico gerado, os dois eixos estão correlacionados? Justifique.


É possível observar que quanto menor o preço os produtos tendem a ser mais pedidos.